In [ ]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import mne

#Load data, filter, make folders

from main_meg_qc import initial_stuff

In [ ]:

#Passing the whole config:
#n_events, df_epochs_mags, df_epochs_grads, epochs_mags, epochs_grads, mags, grads, filtered_d, filtered_d_resamp, raw_cropped, raw=initial_stuff(duration, config=config)
#Passing separate parameters from config:
#n_events, df_epochs_mags, df_epochs_grads, epochs_mags, epochs_grads, mags, grads, filtered_d, filtered_d_resamp, raw_cropped, raw=initial_stuff(duration, data_file)

n_events, df_epochs_mags, df_epochs_grads, epochs_mags, epochs_grads, mags, grads, channels, filtered_d, filtered_d_resamp, raw_cropped, raw=initial_stuff(sid='1')

In [ ]:
def RMSE(data_m_or_g: np.array or list) -> np.array:
    ''' RMSE - general root means squared error function to use in other functions of this module.
    Alternatively std could be used, but the calculation time of std is lower, result is the same.
    
    Args:
    data_m_or_g (np.array or list): data for magnetometer or gradiometer given as np array or list 
        (it can be 1 channel or several as 2 dimentional array or list of lists
        
    Returns:
    rmse_np (np.array): rmse as numpy array (1 if 1 channel was given, more if more channels)'''

    data_m_or_g=np.array(data_m_or_g) #convert to numpy array if it s not
    rmse_list=[]

    data_minentions=len(data_m_or_g.shape)
    if data_minentions==2: #if the data has raws and columns - iterate over raws. if input is 1 dimentional - juat calculate the whole thing.
        for i, dat_raw in enumerate (data_m_or_g):
            y_actual=dat_raw
            y_pred=y_actual.mean()
            rmse_data=np.sqrt(((y_pred - y_actual) ** 2).mean())
            rmse_list.append(rmse_data)
    elif data_minentions==1:
        y_actual=data_m_or_g
        y_pred=data_m_or_g.mean()
        rmse_data=np.sqrt(((y_pred - y_actual) ** 2).mean())
        rmse_list.append(rmse_data)
    else:
        print('Only 1 or 2 dimentional data is accepted, not more!')
        return

    rmse_np=np.array(rmse_list) #conver to numpy array

    return rmse_np

In [ ]:
def RMSE_meg_all_OLD(data: mne.io.Raw, mags: list, grads: list, std_lvl: int) -> list: 

    '''Root mean squared error calculated over ALL data (not epoched)
    
    Args:
    data (mne.raw): data in raw format
    mags (list of tuples): magnetometer channel name + its index
    grads (list of tuples): gradiometer channel name + its index
    std_lvl (int): how many standard deviations from the mean are acceptable in the data. 
        data variability over this setting will be concedered as too too noisy, under -std_lvl as too flat 
    
    Returns:
    m_big_std_with_value (list of tuples): list of magnetometers with too high std value, 
    g_big_std_with_value (list of tuples): list of gradiometers with too high std value, 
    m_small_std_with_value (list of tuples): list of magnetometers with too low std value
    g_small_std_with_value (list of tuples): list of gradiometers with too high std value
    std_mags (np.array): std values for magnetometers
    std_grads (np.array): std values for gradiometers
    '''

    # Separate data for mags and grads in 2 arrays.
    selected_mags = [item[1] for item in mags]
    selected_grads = [item[1] for item in grads]
    data_mags, _ = data[selected_mags, :]  
    data_grads, _ = data[selected_grads, :]  

    # %% Calculate STD or RMSE of each channel

    #Calculate RMSE for each channel (separated mags and grads) - for the entire time duration:
    std_mags = RMSE(data_mags)
    std_grads = RMSE(data_grads)

    #STD (if wanna use insted of RMSE. it will exactly replace the RMSE function above):
    #std_mags=np.std(data_mags, axis=1) #calculate std of all magnetometers (along second dimantion)
    #std_grads=np.std(data_grads, axis=1) #calculate std of all gradiometers (along second dimantion)


    # Check if channel data is within std_lvl over all channels.

    std_std_mags=np.std(std_mags)
    std_std_grads=np.std(std_grads)

    mean_std_mags=np.mean(std_mags)
    mean_std_grads=np.mean(std_grads)

    ch_ind_large_std_mags= np.where(std_mags > mean_std_mags+std_lvl*std_std_mags) #find magn channels with largest std
    ch_ind_large_std_grads= np.where(std_grads > mean_std_grads+std_lvl*std_std_grads) 
    ch_ind_small_std_mags= np.where(std_mags < mean_std_mags-std_lvl*std_std_mags) #find magn channels with smallest std
    ch_ind_small_std_grads= np.where(std_grads < mean_std_grads-std_lvl*std_std_grads)

    magn_channel_big_std=np.array(mags)[ch_ind_large_std_mags] #find the name of the magn with largest std 
    grad_channel_big_std=np.array(grads)[ch_ind_large_std_grads]
    magn_channel_small_std=np.array(mags)[ch_ind_small_std_mags]
    grad_channel_small_std=np.array(grads)[ch_ind_small_std_grads]


    #This function simply makes a list of tuples. Each tuple is: name of channel, std value.
    #Each tuple represents channel with too big or too small std, calculated over whole data.

    def Channels_with_nonnormal_stds(ch_ind, all_stds_m_or_g, channels_big_std_names):
        channel_big_std_vals=all_stds_m_or_g[ch_ind]
        nonnormal_std_with_value=[]
        for id, val in enumerate (ch_ind[0]):
            new_tuple=(channels_big_std_names[id][0],  channel_big_std_vals[id])
            nonnormal_std_with_value.append(new_tuple)
        return(nonnormal_std_with_value)

    m_big_std_with_value=Channels_with_nonnormal_stds(ch_ind_large_std_mags, std_mags, magn_channel_big_std)
    g_big_std_with_value=Channels_with_nonnormal_stds(ch_ind_large_std_grads, std_grads, grad_channel_big_std)
    m_small_std_with_value=Channels_with_nonnormal_stds(ch_ind_small_std_mags, std_mags, magn_channel_small_std)
    g_small_std_with_value=Channels_with_nonnormal_stds(ch_ind_small_std_grads, std_grads, grad_channel_small_std)
        
    #Return the channel names with STD over the set STD level and under the set negative STD level.
    return(m_big_std_with_value, g_big_std_with_value, m_small_std_with_value, g_small_std_with_value, std_mags, std_grads)


In [ ]:
#Try: USING RESAMPLED DATA HERE

m_big_std_with_value, g_big_std_with_value, m_small_std_with_value, g_small_std_with_value, rmse_mags, rmse_grads=RMSE_meg_all_OLD(data=filtered_d_resamp, 
    mags=mags, grads=grads, std_lvl=1)

print('Magnetometers with high std:')
print(m_big_std_with_value)

print('Gradiometers with high std:')
print(g_big_std_with_value)

In [ ]:
def RMSE_meg_all(data: mne.io.Raw, channels: list, std_lvl: int) -> list: 

    '''Root mean squared error calculated over ALL data (not epoched)
    
    Args:
    data (mne.raw): data in raw format
    channels (list of tuples): list of channel names + their indexes (mags or grads)
    std_lvl (int): how many standard deviations from the mean are acceptable in the data. 
        data variability over this setting will be concedered as too too noisy, under -std_lvl as too flat 
    
    Returns:
    big_std_with_value (list of tuples): list of channels with too high std value, 
    small_std_with_value (list of tuples): list of channels with too low std value
    std_channels (np.array): std values for channels
    '''

    # Separate data for mags and grads in 2 arrays.
    selected_channels = [ch[1] for ch in channels]
    data_channels, _ = data[selected_channels, :]  

    # %% Calculate STD or RMSE of each channel

    #Calculate RMSE for each channel (separated mags and grads) - for the entire time duration:
    std_channels = RMSE(data_channels)

    #STD (if wanna use insted of RMSE. it will exactly replace the RMSE function above):
    #std_channels=np.std(data_channels, axis=1) 

    # Check if channel data is within std_lvl over all channels.
    std_std_channels=np.std(std_channels)
    mean_std_channels=np.mean(std_channels)

    ch_ind_large_std= np.where(std_channels > mean_std_channels+std_lvl*std_std_channels) #find channels with largest std
    ch_ind_small_std= np.where(std_channels < mean_std_channels-std_lvl*std_std_channels) #findchannels with smallest std

    channel_big_std=np.array(channels)[ch_ind_large_std] #find the name of the channel with largest std 
    channel_small_std=np.array(channels)[ch_ind_small_std]


    def Channels_with_nonnormal_stds(ch_ind, all_stds_m_or_g, channels_big_std_names):
        #This function simply makes a list of tuples. Each tuple is: name of channel, std value.
        #Each tuple represents channel with too big or too small std, calculated over whole data.
        channel_big_std_vals=all_stds_m_or_g[ch_ind]
        nonnormal_std_with_value=[]
        for id, val in enumerate (ch_ind[0]):
            new_tuple=(channels_big_std_names[id][0],  channel_big_std_vals[id])
            nonnormal_std_with_value.append(new_tuple)
        return(nonnormal_std_with_value)

    big_std_with_value=Channels_with_nonnormal_stds(ch_ind_large_std, std_channels, channel_big_std)
    small_std_with_value=Channels_with_nonnormal_stds(ch_ind_small_std, std_channels, channel_small_std)
        
    #Return the channel names with STD over the set STD level and under the set negative STD level.
    return(big_std_with_value, small_std_with_value, std_channels)

In [ ]:
#Try: USING RESAMPLED DATA HERE

m_big_std_with_value, m_small_std_with_value, rmse_mags=RMSE_meg_all(data=filtered_d_resamp, 
    channels=mags, std_lvl=1)
print('Magnetometers with high std:')
print(m_big_std_with_value)

g_big_std_with_value, g_small_std_with_value, rmse_grads=RMSE_meg_all(data=filtered_d_resamp, 
    channels=grads, std_lvl=1)
print('\n Gradiometers with high std:')
print(g_big_std_with_value)

In [ ]:
def boxplot_std_hovering_plotly(std_data: list, tit: str, channel_names: list, sid: str):

  '''Creates representation of calculated std data as a boxplot (box containd magnetometers or gradiomneters, not together): 
  each dot represents 1 channel: name: std value over whole data of this channel. Too high/low stds are outliers.
  Implemebted with plotly: https://plotly.github.io/plotly.py-docs/generated/plotly.graph_objects.Box.html
  The figure will be saved as an interactive html file.

  Args:
  std_data (list): stds for mags or grads calculated in RMSE_meg_all, 
  tit (str): title, like "Magnetometers", or "Gradiometers", 
  channel_names (list of tuples): magnetometer channel name + its index, 
  sid (str): subject id number, like '1'

  Returns:
  fig (go.Figure): plottly figure
  fig_path (str): path where the figure is saved as html file
  '''

  unit='?'
  if tit=='Magnetometers':
    unit='T'
  elif tit=='Gradiometers':
    unit='T/m'
  else:
    print('Please check tit input. Has to be "Magnetoneters" or "Gradiometers"')

  ch_names=[m[0] for m in channel_names] #names of channels for annotating the plot
  df = pd.DataFrame (std_data, index=ch_names, columns=['std'])

  fig = go.Figure()

  fig.add_trace(go.Box(x=df['std'],
  name="",
  text=df['std'].index, 
  opacity=0.7, 
  boxpoints="all", 
  pointpos=0,
  marker_size=5,
  line_width=1))
  fig.update_traces(hovertemplate='%{text}'+'<br>STD: %{x: .0f}')
      

  fig.update_layout(
      yaxis={'visible': False, 'showticklabels': False},
      xaxis = dict(
      showexponent = 'all',
      exponentformat = 'e'),
      xaxis_title="standard deviation in "+unit,
      title={
      'text': "Standard deviation of the data for "+tit,
      'y':0.85,
      'x':0.5,
      'xanchor': 'center',
      'yanchor': 'top'})
      
  fig.show()

  fig_name='Stds_all_data_'+tit+'.html'
  fig_path='../derivatives/sub-'+sid+'/megqc/figures/'+fig_name
  fig.write_html(fig_path)

  return(fig, fig_path)

In [ ]:
fig_m, fig_path_m=boxplot_std_hovering_plotly(std_data=rmse_mags, tit='Magnetometers', channel_names=mags, sid='1')

In [ ]:
fig_g, fig_path_g=boxplot_std_hovering_plotly(std_data=rmse_grads, tit='Gradiometers', channel_names=grads, sid='1')



In [ ]:
def std_mg_old(mg_names: list, df_mg: pd.DataFrame, epoch_numbers: list) -> pd.DataFrame:

    '''Calculate std for every separate epoch of mags or grads.
    Used as internal function in RMSE_meg_epoch

    Args:
    mg_names (list of tuples): channel name + its index, 
    df_mg (pd.DataFrame): data frame containing data for all epochs for mags or for grads
    epoch_numbers (list): list of epoch numbers

    Returns:
    df_std_mg (pd.DataFrame): data frame containing stds for all epoch for each channel
    '''
    
    dict_mg = {}

    for ep in epoch_numbers: #loop over each epoch
        rows_for_ep = [row for row in df_mg.iloc if row.epoch == ep] #take all rows of 1 epoch, all channels.
        #std_epoch = [] #list with stds
        rmse_epoch=[]

        for ch_name in mg_names: #loop over channel names
            data_ch_epoch = [row_mg[ch_name] for row_mg in rows_for_ep] #take the data for 1 epoch for 1 channel
            rmse_ch_ep = RMSE(data_ch_epoch)
            rmse_ch_ep=np.float64(rmse_ch_ep) #convert from ndarray to float
            rmse_epoch.append(rmse_ch_ep)

            #std_ch_ep = np.std(data_ch_epoch) #if want to use std instead
            

        dict_mg[ep] = rmse_epoch

    df_std_mg = pd.DataFrame(dict_mg, index=mg_names)

    return(df_std_mg)

In [ ]:
def std_mg(mg_names: list, df_mg: pd.DataFrame, epoch_numbers: list) -> pd.DataFrame:

    '''Calculate std for every separate epoch of mags or grads.
    Used as internal function in RMSE_meg_epoch

    Args:
    mg_names (list of tuples): channel name + its index, 
    df_mg (pd.DataFrame): data frame containing data for all epochs for mags or for grads
    epoch_numbers (list): list of epoch numbers

    Returns:
    df_std_mg (pd.DataFrame): data frame containing stds for all epoch for each channel
    '''
    
    dict_mg = {}

    for ep in epoch_numbers: #loop over each epoch
        #rows_for_ep = [row for row in df_mg.iloc if row.epoch == ep] #take all rows of 1 epoch, all channels.
        df_one_ep=df_mg.loc[df_mg['epoch'] == ep]
        #std_epoch = [] #list with stds
        rmse_epoch=[]

        for ch_name in mg_names: #loop over channel names
            data_ch_epoch=list(df_one_ep.loc[:,ch_name])
            #data_ch_epoch = [row_mg[ch_name] for row_mg in rows_for_ep] #take the data for 1 epoch for 1 channel
            rmse_ch_ep = RMSE(data_ch_epoch)
            rmse_ch_ep=np.float64(rmse_ch_ep) #convert from ndarray to float
            rmse_epoch.append(rmse_ch_ep)

            #std_ch_ep = np.std(data_ch_epoch) #if want to use std instead
            

        dict_mg[ep] = rmse_epoch

    df_std_mg = pd.DataFrame(dict_mg, index=mg_names)

    return(df_std_mg)

In [ ]:
# STD over epochs: use 2 separate data frames for mags and grads in calculations:

def RMSE_meg_epoch_OLD(mags: list, grads: list, std_lvl: int, n_events: int, df_epochs_mags: pd.DataFrame, df_epochs_grads:pd.DataFrame, sid: str):

    '''
    - Calculate std for every separate epoch of mags + grads
    - Find which channels in which epochs have too high/too small stds
    - Extract all these data for the user as csc files.

    Args:
    mags (list of tuples): magnetometer channel name + its index,
    grads (list of tuples): gradiometer channel name + its index,
    std_lvl (int): how many standard deviations from the mean are acceptable in the data. 
        data variability over this setting will be concedered as too too noisy, under -std_lvl as too flat 
    n_events(int): number of events, 
    df_epochs_mags (pd.DataFrame): data frame containing stds for all epoch for each magnetometer
    df_epochs_grads(pd.DataFrame): data frame containing stds for all epoch for each gradiometer
    sid (str): subject id number, like '1'

    Returns:
    df_std_mags(pd.DataFrame): data frame containing std data for each epoch, each channel in mags
    df_std_grads(pd.DataFrame): data frame containing std data for each epoch, each channel in grads
    + csv files of these two
    '''

    # 1) Loop over the epochs of each channel and check for every separate magn and grad and calculate std
    
    eps=list(range(0,n_events)) #list of epoch numbers
   
    mags_names = [mag[0] for mag in mags]
    grads_names = [grad[0] for grad in grads]


    #Apply function from above for mags and grads:
    df_std_mags=std_mg(df_mg=df_epochs_mags, mg_names=mags_names, epoch_numbers=eps)
    df_std_grads=std_mg(df_mg=df_epochs_grads, mg_names=grads_names, epoch_numbers=eps)

    # 2) Check (which epochs for which channel) are over 1STD (or 2, 3, ets STDs) for (this epoch for all channels)

    #Find what is 1 std over all channels per 1 epoch:
    std_std_mags_per_epoch=[]
    std_std_grads_per_epoch=[]
    mean_std_mags_per_epoch=[]
    mean_std_grads_per_epoch=[]

    for ep in eps: #goes over each epoch
        std_std_mags_per_epoch.append(np.std(df_std_mags.iloc[:, ep])) #std of stds of all channels of every single epoch
        std_std_grads_per_epoch.append(np.std(df_std_grads.iloc[:, ep]))

        mean_std_mags_per_epoch.append(np.mean(df_std_mags.iloc[:, ep])) #mean of stds of all channels of every single epoch
        mean_std_grads_per_epoch.append(np.mean(df_std_grads.iloc[:, ep]))


    df_ch_ep_large_std_mags=df_std_mags.copy()
    df_ch_ep_large_std_grads=df_std_grads.copy()

    df_ch_ep_small_std_mags=df_std_mags.copy()
    df_ch_ep_small_std_grads=df_std_grads.copy()

    #Now see which channles in epoch are over 1 std or under -1 std:
    for ep in eps: #goes over each epoch   
        df_ch_ep_large_std_mags.iloc[:,ep] = df_ch_ep_large_std_mags.iloc[:,ep] > mean_std_mags_per_epoch[ep]+std_lvl*std_std_mags_per_epoch[ep] #magnetometers
        df_ch_ep_large_std_grads.iloc[:,ep] = df_ch_ep_large_std_grads.iloc[:,ep] > mean_std_grads_per_epoch[ep]+std_lvl*std_std_grads_per_epoch[ep] #gradiometers

        df_ch_ep_small_std_mags.iloc[:,ep] = df_ch_ep_small_std_mags.iloc[:,ep] < mean_std_mags_per_epoch[ep]-std_lvl*std_std_mags_per_epoch[ep] #magnetometers
        df_ch_ep_small_std_grads.iloc[:,ep] = df_ch_ep_small_std_grads.iloc[:,ep] < mean_std_grads_per_epoch[ep]-std_lvl*std_std_grads_per_epoch[ep] #gradiometers


    # Create csv files  for the user:

    df_std_mags.to_csv('../derivatives/sub-'+sid+'/megqc/csv files/std_mags_per_epoch.csv')
    
    df_std_grads.to_csv('../derivatives/sub-'+sid+'/megqc/csv files/std_grads_per_epoch.csv')

    df_ch_ep_large_std_mags.to_csv('../derivatives/sub-'+sid+'/megqc/csv files/Large_std_mags_per_epoch.csv')

    df_ch_ep_large_std_grads.to_csv('../derivatives/sub-'+sid+'/megqc/csv files/Large_std_grads_per_epoch.csv')

    df_ch_ep_small_std_mags.to_csv('../derivatives/sub-'+sid+'/megqc/csv files/Small_std_mags_per_epoch.csv')

    df_ch_ep_small_std_grads.to_csv('../derivatives/sub-'+sid+'/megqc/csv files/Small_std_grads_per_epoch.csv')

    return(df_std_mags, df_std_grads)

In [ ]:
#try (will output csv files):
# USING NON RESAMPLED DATA

df_std_mags, df_std_grads=RMSE_meg_epoch_OLD(mags=mags, grads=grads, std_lvl=1, n_events=n_events, df_epochs_mags=df_epochs_mags, df_epochs_grads=df_epochs_grads, sid='1') 


In [ ]:
# STD over epochs: use 2 separate data frames for mags and grads in calculations:

def RMSE_meg_epoch(ch_type: str, channels: list, std_lvl: int, n_events: int, df_epochs: pd.DataFrame, sid: str):

    '''
    - Calculate std for every separate epoch of mags + grads
    - Find which channels in which epochs have too high/too small stds
    - Extract all these data for the user as csc files.

    Args:
    channels (list of tuples): channel name + its index as list,
    std_lvl (int): how many standard deviations from the mean are acceptable in the data. 
        data variability over this setting will be concedered as too too noisy, under -std_lvl as too flat 
    n_events(int): number of events, 
    df_epochs (pd.DataFrame): data frame containing stds for all epoch for each channel
    sid (str): subject id number, like '1'

    Returns:
    df_std_epoch(pd.DataFrame): data frame containing std data for each epoch, each channel 
    + csv file of this df
    '''

    # 1) Loop over the epochs of each channel and check for every separate magn and grad and calculate std
    
    eps=list(range(0,n_events)) #list of epoch numbers
   
    ch_names = [ch[0] for ch in channels]

    #Apply function from above for mags and grads:
    df_std=std_mg(df_mg=df_epochs, mg_names=ch_names, epoch_numbers=eps)

    # 2) Check (which epochs for which channel) are over 1STD (or 2, 3, etc STDs) for this epoch for all channels

    #Find what is 1 std over all channels per 1 epoch:
    std_std_per_epoch=[]
    mean_std_per_epoch=[]

    for ep in eps: #goes over each epoch
        std_std_per_epoch.append(np.std(df_std.iloc[:, ep])) #std of stds of all channels of every single epoch
        mean_std_per_epoch.append(np.mean(df_std.iloc[:, ep])) #mean of stds of all channels of every single epoch

    df_ch_ep_large_std=df_std.copy()
    df_ch_ep_small_std=df_std.copy()

    #Now see which channles in epoch are over 1 std or under -1 std:
    for ep in eps: #goes over each epoch   
        df_ch_ep_large_std.iloc[:,ep] = df_ch_ep_large_std.iloc[:,ep] > mean_std_per_epoch[ep]+std_lvl*std_std_per_epoch[ep] 
        df_ch_ep_small_std.iloc[:,ep] = df_ch_ep_small_std.iloc[:,ep] < mean_std_per_epoch[ep]-std_lvl*std_std_per_epoch[ep] 


    # Create csv files  for the user:
    df_std.to_csv('../derivatives/sub-'+sid+'/megqc/csv files/std_per_epoch_'+ch_type+'.csv')
    df_ch_ep_large_std.to_csv('../derivatives/sub-'+sid+'/megqc/csv files/Large_std_per_epoch_'+ch_type+'.csv')
    df_ch_ep_small_std.to_csv('../derivatives/sub-'+sid+'/megqc/csv files/Small_std_per_epoch_'+ch_type+'.csv')

    return df_std

In [ ]:
df_std_mags = RMSE_meg_epoch(ch_type='mags', channels=mags, std_lvl=1, n_events=n_events, df_epochs=df_epochs_mags, sid='1') 
df_std_grads = RMSE_meg_epoch(ch_type='grads', channels=grads, std_lvl=1, n_events=n_events, df_epochs=df_epochs_grads, sid='1') 

In [ ]:
#plot using new universal plotting function:

from universal_plots import boxplot_channel_epoch_hovering_plotly

fig_std_epoch_m, fig_path_m_std_epoch = boxplot_channel_epoch_hovering_plotly(df_mg=df_std_mags, ch_type='Magnetometers', sid='1', what_data='stds')

fig_std_epoch_g, fig_path_g_std_epoch =boxplot_channel_epoch_hovering_plotly(df_mg=df_std_grads, ch_type='Gradiometers', sid='1', what_data='stds')

fig_std_epoch_m.show()
fig_std_epoch_g.show()

In [ ]:
from universal_html_report import make_std_peak_report

list_of_figure_paths=[fig_path_m, fig_path_g, fig_path_m_std_epoch, fig_path_g_std_epoch]
make_std_peak_report(sid='1', what_data='stds', list_of_figure_paths=list_of_figure_paths)

In [ ]:
#%%
def MEG_QC_rmse(sid: str, config, channels: dict, m_or_g_title: dict, df_epochs:pd.DataFrame, filtered_d_resamp, n_events: int, m_or_g_chosen):

    from universal_plots import boxplot_channel_epoch_hovering_plotly
    from universal_html_report import make_std_peak_report

    rmse_section = config['RMSE']

    # import RMSE_meg_qc as rmse #or smth like this - when it's extracted to .py
    std_lvl = rmse_section.getint('std_lvl')

    list_of_figure_paths = []
    list_of_figure_paths_std_epoch = []
    big_std_with_value = {}
    small_std_with_value = {}
    fig = {}
    fig_path = {}
    df_std = {}
    fig_std_epoch = {}
    fig_path_std_epoch = {}
    rmse = {}

    # will run for both if mags and grads both chosen,otherwise just for one of them:
    for m_or_g in m_or_g_chosen:
        big_std_with_value[m_or_g], small_std_with_value[m_or_g], rmse[m_or_g] = RMSE_meg_all(data=filtered_d_resamp, channels=channels[m_or_g], std_lvl=1)

        fig[m_or_g], fig_path[m_or_g] = boxplot_std_hovering_plotly(std_data=rmse[m_or_g], tit=channels[m_or_g], channels=channels[m_or_g], sid=sid)
        
        df_std[m_or_g] = RMSE_meg_epoch(ch_type=m_or_g, channels=channels[m_or_g], std_lvl=std_lvl, n_events=n_events, df_epochs=df_epochs[m_or_g], sid=sid) 

        fig_std_epoch[m_or_g], fig_path_std_epoch[m_or_g] = boxplot_channel_epoch_hovering_plotly(df_mg=df_std[m_or_g], ch_type=m_or_g, sid=sid, what_data='stds')
        
        list_of_figure_paths.append(fig_path[m_or_g])
        list_of_figure_paths_std_epoch.append(fig_path_std_epoch[m_or_g])
    
    list_of_figure_paths += list_of_figure_paths_std_epoch

    make_std_peak_report(sid=sid, what_data='stds', list_of_figure_paths=list_of_figure_paths)



In [ ]:
!jupyter nbconvert RMSE_meq_qc.ipynb --to python

### Heading